In [2]:
from dotenv import load_dotenv
import os
import geopandas as gpd

load_dotenv()

True

In [3]:
from minio import Minio

minio_client = Minio(
	endpoint=os.getenv('S3_ENDPOINT'),
	access_key=os.getenv('ACCESS_KEY_ID'),
	secret_key=os.getenv('SECRET_ACCESS_KEY'),
	secure=True,
)

bucket = os.getenv('BUCKET')

In [4]:
import hashlib
import requests

def calculate_checksum(file_path):
    sha1_hash = hashlib.sha1()
    response = requests.get(file_path, stream=True)
    for chunk in response.iter_content(chunk_size=4096):
        if chunk:
            sha1_hash.update(chunk)
    return sha1_hash.hexdigest()

In [25]:
from datetime import datetime
from shapely.geometry import Polygon
from minio.commonconfig import REPLACE, CopySource
from tqdm import tqdm

# list files in bucket

# dataset_id = "67d1a16a82ec193cb1942a4d" # imagine
dataset_id = "67d1a13482ec193cb1942a4b" # orbitalai

files = list(minio_client.list_objects(bucket, prefix=dataset_id, recursive=True))

size = 0
count = 1
data = []
for file in tqdm(files):
# for file in minio_client.list_objects(bucket, prefix=dataset_id, recursive=True):
	# print(file.object_name)
	stats = minio_client.stat_object(bucket, file.object_name)
	if stats.size == 0 or file.object_name.endswith('catalog.v1.parquet'):
		continue
	size += stats.size
	
	if file.object_name.endswith('_1'):
		print(file.object_name)
		break
		# try:
		# 	new_name = file.object_name[:-2]
		# 	stats = minio_client.stat_object(bucket, new_name)
		# 	size += stats.size
		# 	minio_client.remove_object(bucket, file.object_name)
		# except:
		# 	# print(f"Renaming {file.object_name} to {new_name}")
		# 	# Copy object with new name
		# 	# minio_client.copy_object(bucket, new_name, CopySource(bucket, file.object_name) )
		# 	print(f"{new_name} not found")

	item_id = file.object_name.split(f'{dataset_id}/')[-1]
	# presigned_url = minio_client.presigned_get_object(bucket, file.object_name)
	stac_item  = {
		'type': 'Feature',
		'stac_version': '1.0.0',
		'stac_extensions': [],
		'datetime': datetime.now(),  # must be native timestamp (https://github.com/apache/parquet-format/blob/master/LogicalTypes.md#timestamp)
		'id': item_id,
		'bbox': {
			'xmin': 0.0,
			'ymin': 0.0,
			'xmax': 0.0,
			'ymax': 0.0
		}, # infer from file or from list of geometries
		'geometry': Polygon(), # empty polygon
		'assets': { 'asset': { # STAC needs this to be a Dict[str, Asset], not list !!! use same key or parquet breaks !!!
			'href': f'https://api.eotdl.com/models/{dataset_id}/stage/{item_id}', 
			'checksum': "123", #calculate_checksum(presigned_url),
			'timestamp': datetime.now(),
			'size': stats.size,
		}},
		"links": [],
		# 'collection': 'source',
		# anything below are properties (need at least one!)
		'repository': 'eotdl',				
	}
	data.append(stac_item)

	count += 1

	# if count > 10:
	# 	break

	# if count % 20 == 0:
	# 	print(f"Processed {count} files")

print(size)


100%|██████████| 491/491 [00:46<00:00, 10.45it/s]

558476801030


In [26]:
gdf = gpd.GeoDataFrame(data, geometry='geometry')
catalog_name = f'catalog.v1.parquet'
gdf.to_parquet(catalog_name)

In [27]:
# gdf.assets.apply(lambda x: x['asset']['size'])
gdf

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-04-03 15:48:26.908544,167900-9324410_32725_2021-05-25T12-53-38.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
1,Feature,1.0.0,[],2025-04-03 15:48:27.002286,180110-8219120_32733_2022-05-20T09-35-07.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
2,Feature,1.0.0,[],2025-04-03 15:48:27.096207,180820-1211940_32649_2022-01-07T03-24-52.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
3,Feature,1.0.0,[],2025-04-03 15:48:27.188937,200330-489270_32622_2020-10-16T14-11-10.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
4,Feature,1.0.0,[],2025-04-03 15:48:27.276668,202370-435560_32621_2020-11-11T14-31-34.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
...,...,...,...,...,...,...,...,...,...,...
484,Feature,1.0.0,[],2025-04-03 15:49:13.243421,787500-9268390_32720_2020-07-11T14-24-39.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
485,Feature,1.0.0,[],2025-04-03 15:49:13.334232,791410-9177750_32736_2021-10-25T08-02-13.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
486,Feature,1.0.0,[],2025-04-03 15:49:13.416651,792560-3019350_32649_2022-10-11T03-10-16.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl
487,Feature,1.0.0,[],2025-04-03 15:49:13.526270,793640-606050_32619_2020-07-13T15-01-14.zip,"{'xmin': 0.0, 'ymin': 0.0, 'xmax': 0.0, 'ymax'...",POLYGON EMPTY,{'asset': {'href': 'https://api.eotdl.com/mode...,[],eotdl


In [28]:
minio_client.fput_object(bucket, f"{dataset_id}/catalog.v1.parquet", catalog_name)